## Uploading many spectra

In [ ]:
import os
import sys
import json
import requests
from requests.auth import HTTPBasicAuth
import base64

# Optional (if reading from a .csv file)
import numpy as np

# Optional (if coordinates not in decimal degrees)
from astropy import units as u
from astropy.coordinates import SkyCoord

### 1. Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example, in an open shell terminal type: <code>export SLED_USERNAME=<your_SLED_username></code> and <code>export SLED_PASSWORD=<your_SLED_password></code>. You can also include these two commands in your shell profile file (e.g. ~/.bash_profile in Unix/Linux/Mac) or in an environment file.

In [ ]:
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')
if (not username) or (not password):
    print('You have not set your SLED username and password environment variables.')

### 2. Define a list of spectrum dictionaries

All the fields associated with spectral data are described in the [SLED guide](https://sled.amnh.org/sled_guide/#object-specs-spectrum). In addition to these fields, the RA and DEC of the associated lens must be given. Coordinates should be in decimal degrees and in the J2000 system. To convert from ra/dec in hours-minutes/degrees-minutes, you can use the astropy coords functionality. The associated plot of the spectrum can be in .png, .jpg, or .jpeg format. It is recommended to restrict access to the uploaded data by setting 'access_level'='PRI' (or just omitting this field).

In [ ]:
# An imaging data object. The RA and DEC are required only to match it to the correct lens.
spectrum_tmp = {
    'ra': 0.0,  # Decimal field, up to 5 decimal places, between 0 and 360
    'dec': 0.0,  # Decimal field, up to 5 decimal places, between -90 and 90
    'instrument': '',  # String, pre-defined choices, see guide
    'image': '',  # The name of the image file, this will be replaced by the actual image data further down
    'future': 'False', # String, 'True' or 'False'
    'date_taken': '1990-03-22', # A date in the format YYYY-MM-DD, can be in the future
    'access_level': 'PRI'
}


# Here we have the RA, DEC, and an image file name as lists:
ras = [0.1,0.2] # dummy values
decs = [-10.1,-10.2] # dummy values
dates = ['1990-03-22','2001-04-01']
fnames = ['spectrum_0.png','spectrum_1.png'] # dummy values

## If reading from a .csv file use the following command (assuming 'instrument' doesn't change):
#ras,decs,dates,fnames = np.genfromtxt('random_file.csv',dtype=None,skip_header=1,usecols=[0,1,3,4],delimiter=',',encoding=None,unpack=True)

# And a path where our images are stored:
path_to_img = 'some/dummy/path/'



# We define a list of spectrum objects:
new_spectra = []
for i in range(0,len(ras)):
    spectrum = spectrum_tmp.copy()

    spectrum['image'] = path_to_img + fnames[i]
    spectrum['ra'] = round(ras[i],5)
    spectrum['dec'] = round(decs[i],5)
    spectrum['date_taken'] = dates[i]
    
    ## If the RA and DEC are not in decimal degrees then use the following lines instead:
    #c1 = SkyCoord(ras[i], unit=(u.hourangle, u.deg))
    #c2 = SkyCoord(decs[i], unit=(u.hourangle, u.deg))
    #spectrum['ra'] = round(c1,5)
    #spectrum['dec'] = round(c2,5)
    
    if not os.path.isfile(spectrum['image']):
        print(i,': NO image found with this filename: ',spectrum['image'])
        sys.exit()
    else:
        new_spectra.append(spectrum)

        
print('Spectrum objects to be injected: ',len(new_spectra))
if len(new_spectra)>0:
    print('First spectrum in the list looks like this: ')
    print(new_spectra[0])
else:
    print('No spectra to inject!')
    sys.exit()

### 3. Now send request to the database

Use the requests package to send a POST request to the relevant API endpoint. The *'image'* field is replaced by the actual image data in base64 encoding. There is no need to change the following lines.


In [ ]:
for spectrum in new_spectra:
    if spectrum['future'] == 'False':
        spectrum['image'] = image_to_base64(f"{spectrum['image']}")

final_json = {
    'data_type': 'spectrum',
    'data': new_spectra    
}
        
print("Uploading to the SLED server...")
r = requests.post("https://sled.amnh.org/api/upload-data/",
                  data=json.dumps(final_json),
                  headers={'Content-Type':'application/json','User-Agent':'SLED-api/1.0'},
                  auth=HTTPBasicAuth(username,password)
                  )
dbquery = json.loads(r.text)

### 4. Understanding the response

There are three possible types of response from the SLED API:
- *errors*: your uploaded sample contains errors, e.g. the RA,DEC are not in the right format, etc.
- *further action required*: if public (PUB) data are uploaded there will be some further action required.
- *success*: the sample of new data was uploaded successfully

In [ ]:
print(dbquery)